In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dest_path = '/kaggle/input/pfizer-vaccine-tweets/vaccination_tweets.csv'
df = pd.read_csv(dest_path)
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.set_index('id', inplace=True)
df.head()

In [ ]:
df['time'] = pd.to_datetime(df['date'])
df.drop(columns={'date'}, inplace=True)
df.info()

In [ ]:
df.head()

In [ ]:
data = pd.read_csv('/kaggle/input/vader-lexicon/vader_lexicon.txt', sep='\t', header=None)
df_sent = pd.DataFrame()
df_sent['token']=data[0]
df_sent['sentiment']=data[1]

df_sent.set_index('token', inplace=True)
df_sent.head()

In [ ]:
df['text'] = df['text'].str.lower()
df.head()

In [ ]:
import re
punct_re = r'[^\w\s\\n]'
regex = re.compile(punct_re, flags = re.IGNORECASE)
df['no_punc'] = df['text'].replace(to_replace=regex, value=' ')

In [ ]:
df['no_punc']

In [ ]:
splitted = df['no_punc'].str.split(expand=True)
stacked = splitted.stack()
tidy_format = stacked.reset_index()
tidy_format = tidy_format.set_index('id')
tidy_format = tidy_format.rename(columns={'level_1': 'num'})
tidy_format.rename(columns = {list(tidy_format)[1]: 'word'}, inplace=True)
tidy_format = tidy_format.rename_axis(None)
tidy_format.head(10)

In [ ]:
df_merged = pd.merge(df, tidy_format, left_index=True, right_index=True)
df_merged.drop(columns=['num'],inplace=True)
df_merged = pd.merge(df_merged, df_sent, left_on='word', right_index=True)
#df_merged
text_to_sent = df_merged.groupby('text').sum()['sentiment']
text_to_sent = text_to_sent.to_frame('polarity')
text_to_sent
df = pd.merge(df, text_to_sent, left_on='text', right_on='text', right_index=True).fillna(0.0)

df

In [ ]:
#print('Most negative tweets:')
ordered=df[['text','polarity']].sort_values(by='polarity', ascending=False)
ordered.tail()

In [ ]:
df.loc[1339227682094059520]['text']

In [ ]:
ordered.head()

In [ ]:
df.loc[1339622618073337856]['text']

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline
plt.style.use('fivethirtyeight')
plt.rcParams['figure.figsize'] = (12, 9)
sns.set()
sns.set_context("talk")

In [ ]:
pfizer = df[df['text'].str.contains('pfizer')]
trump = df[df['text'].str.contains('trump')]

sns.distplot(pfizer['polarity'], hist=True, label='pfizer')
sns.distplot(trump['polarity'], hist=True, label='trump')

plt.legend(title='');

In [ ]:
science = df[df['text'].str.contains('science')]
fake = df[df['text'].str.contains('fake')]

sns.distplot(science['polarity'], hist=True, label='science')
sns.distplot(fake['polarity'], hist=True, label='fake')

plt.legend(title='');

In [ ]:
sns.distplot(df['polarity'], hist=True, label='everything')
plt.legend(title='');

In [ ]:
import datetime
def year_fraction(date):
    start = datetime.date(date.year, 1, 1).toordinal()
    year_length = datetime.date(date.year+1, 1, 1).toordinal() - start
    return date.year + float(date.toordinal() - start) / year_length


df['year'] = df['time'].apply(year_fraction)

In [ ]:
df['year'].astype(int).unique()

In [ ]:
sns.distplot(df['year'], hist=True)
plt.legend(title='');
plt.xticks([2020.9,2021.1], ['December', 'February']);

In [ ]:
df['date'] = df['time'].dt.date

In [ ]:
ax=sns.lineplot(x='time', y='polarity', data=df)
ax.set_xticklabels(df['date'],rotation=90);

In [ ]:
df['date'].value_counts().plot(kind="line");